# **Creating a Neural Network for handwritten Digit Recognition with PyTorch**

Author: Javier Eguíluz Romero

This notebook contains the code to create and train an quick neural network that reads images of handwritten digits (from the [MNIST dataset](http://yann.lecun.com/exdb/mnist/)) and gets the actual number. Prior to the data loading and training there're some definitions of functions that will help with the processes.



## Libraries and Settings

In [1]:
import torch
import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch import nn
from torch import optim
from tqdm import tqdm

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Function definitions

### Definition of the Neural Network

In [3]:
# Architecture of the NN. Three linear layers with ReLU activation functions.

class MNIST_NN(nn.Module):
  def __init__(self, size_input, size_output):
    super(MNIST_NN, self).__init__()
    self.fc1 = nn.Linear(size_input, 400)
    self.rel1 = nn.ReLU()
    self.fc2 = nn.Linear(400, 50)
    self.rel2 = nn.ReLU()
    self.fc3 = nn.Linear(50,size_output)

  def forward(self, x):
    x = self.fc1(x)
    x = self.rel1(x)
    x = self.fc2(x)
    x = self.rel2(x)
    x = self.fc3(x)
    return x

### Helper Functions

In [4]:
# Loads the data from the MNIST dataset and prepares it in batches of size "batch_size". If "train_bool" is true, it retrieves the training data, otherwise it gets the testing data.
def LoadData(batch_size, train_bool=True):
  data = DataLoader(
      datasets.MNIST(root='dataset/', train=train_bool, transform=transforms.ToTensor(), download=True),
      batch_size=batch_size, 
      shuffle=True)    
  return data

In [5]:
# Trains the NN taking the training data through as many epochs as defined. The backward method can be changed.
def train(data_train, model, criterion, optimizer, epochs):
  for epoch in tqdm(range(epochs)):
    for data, targets in data_train:
      data = data.to(device)
      targets = targets.to(device)
      data = data.reshape(data.shape[0], -1)

      # forward
      scores = model(data)
      loss = criterion(scores, targets)

      #backward
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

In [6]:
# Provides with the performance of the model via the parameters: Precision, Recall and the F1 score for each one of the classes.
def performance_metrics(model, dataset):
  true_positives = [0]*10
  false_negatives = [0]*10
  false_positives = [0]*10

  model.eval()
  with torch.no_grad():
    for data, targets in dataset:
      data = data.to(device)
      targets = targets.to(device)
      data = data.reshape(data.shape[0], -1)

      scores = model(data)
      _, predicted = scores.max(1)

      for i in range(targets.size()[0]):
        actual_value = targets[i]
        predicted_value = predicted[i]

        if predicted_value == actual_value:
          true_positives[actual_value] += 1
        else:
          false_negatives[actual_value] += 1
          false_positives[predicted_value] += 1

  model.train()

  #precision
  precision = np.divide( true_positives, np.add(true_positives,false_positives) )
  #recall
  recall = np.divide( true_positives, np.add(true_positives,false_negatives) )
  #F1 Score
  F1_score = 2*np.divide( np.multiply(precision,recall), np.add(precision,recall) )

  return precision, recall, F1_score

## Training and evaluation of the Neural Network

### Loading the data from the MNIST Dataset

In [7]:
# Definition of parameters.
batch_size = 64
size_input = 28*28
size_output = 10
learning_rate = 0.001
epochs = 2

In [8]:
# Load the data.
train_data = LoadData(batch_size, train_bool=True)
test_data = LoadData(batch_size, train_bool=False)

### Training of the model

In [9]:
# Specify and train the model
model = MNIST_NN(size_input, size_output).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train(train_data, model, criterion, optimizer, epochs)

100%|██████████| 2/2 [00:27<00:00, 13.86s/it]


### Testing of the model

In [10]:
# Get the performance
precision, recall, F1_score = performance_metrics(model,test_data)
print(f'The performance of the model on the test data gives these averaged values: Precision: {np.mean(precision)*100:.2f}%, Recall: {np.mean(recall)*100:.2f}%, F1 score: {np.mean(F1_score):.4f}.')

The performance of the model on the test data gives these averaged values: Precision: 97.17%, Recall: 97.16%, F1 score: 0.9716.
